In [ ]:
from matplotlib import pyplot
import numpy as np
import os
import torch
import torchvision.transforms.functional as fn
import torchvision
from torchvision.io import read_image
from PIL import Image
from torchvision.transforms import ToTensor
from torch.nn.functional import one_hot as one_hot_encoding
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchsummary import summary
import re
from torch.utils.data import Dataset, DataLoader
from datetime import datetime

In [ ]:
patch = 256
category = {
            'forest': 0, 'buildings': 1, 'river': 2, 'mobilehomepark': 3,
             'harbor': 4, 'golfcourse': 5, 'agricultural': 6, 
             'runway': 7, 'baseballdiamond' : 8, 'overpass' : 9, 'chaparral':10, 
             'tenniscourt':11, 'intersection':12, 'airplane':13, 'parkinglot':14, 
             'sparseresidential':15, 'mediumresidential':16, 'denseresidential':17, 
             'beach':18, 'freeway':19, 'storagetanks':20
}
path = "/Users/summer/desktop/CNNproj-for-landscape-images/data/UCMerced_LandUse/WholeDataset"

In [ ]:
# dataset I created 
# easier to use dataloader 

class LandscapeDataset(Dataset):
    # here label is a 100 element tensor
    def __init__(self, label, img_dir, transform=None, target_transform=None):
        self.img_labels = label
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        path_to_data = self.img_dir + "/" + os.listdir(self.img_dir)[idx]
        image = Image.open(path_to_data)
        
        # make it a tensor
        image = ToTensor()(image)
        label = self.img_labels[idx]
        
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
category = {
            'forest': 0, 'buildings': 1, 'river': 2, 'mobilehomepark': 3,
             'harbor': 4, 'golfcourse': 5, 'agricultural': 6, 
             'runway': 7, 'baseballdiamond' : 8, 'overpass' : 9, 'chaparral':10, 
             'tenniscourt':11, 'intersection':12, 'airplane':13, 'parkinglot':14, 
             'sparseresidential':15, 'mediumresidential':16, 'denseresidential':17, 
             'beach':18, 'freeway':19, 'storagetanks':20
}
path = "/Users/summer/desktop/CNNproj-for-landscape-images/data/UCMerced_LandUse/WholeDataset"

data = os.listdir(path)
labels = torch.zeros(len(data))
# try to find the class that image belongs to
for i in range (len(data)):
    labels[i] = category[re.findall('([a-zA-Z ]+)\d*.*', data[i])[0]]
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize([64,64]),
    # normalize it, get from mean and std from 
    #torchvision.transforms.Normalize([0.4858 , 0.4909, 0.4521],[0.2117, 0.1953, 0.1896]),
    #torchvision.transforms.Normalize([0.9, 0.9, 0.9],[0.2117, 0.1953, 0.1896]),
    
]
)
labels = labels.long()
dataset = LandscapeDataset(
                            labels, 
                           "/Users/summer/desktop/CNNproj-for-landscape-images/data/UCMerced_LandUse/WholeDataset",
                            transform = transform
                          )

In [ ]:
training_set, validation_set, test_set = torch.utils.data.random_split(dataset, 
                                                                       [1260, 420, 420], 
                                                                       generator=torch.Generator().manual_seed(42)
                                                                      )
# Create data loaders for our datasets; shuffle for training, not for validation
training_loader = torch.utils.data.DataLoader(training_set, batch_size=32, shuffle=True,drop_last =True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=32, shuffle=False,drop_last =True)

plt.imshow(dataset[230][0].permute(1,2,0))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, (3, 3),padding="same")
        self.conv2 = nn.Conv2d(32, 64, (3,3),padding='same')
        self.conv3 = nn.Conv2d(64,128,(3,3), padding="same")
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d((2,2), stride=(2,2))
        self.fc = nn.Linear(8192, 21)
        self.dropout = nn.Dropout(p=0.5)
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.flatten(x) # flatten all dimensions except batch
        x = self.fc(x)
        x = self.dropout(x)
        x = self.softmax(x)
        
        return x
    

In [ ]:
def train_one_epoch(epoch_index):
    running_loss = 0.
    last_loss = 0.
    

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)
        

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 10 == 9:
            last_loss = running_loss / 10 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.
    return last_loss

In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
epoch_number = 0
model = Net()
EPOCHS = 100
loss_fn = nn.CrossEntropyLoss()
best_vloss = 1_000_000.
optimizer = torch.optim.Adam(model.parameters())

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number)

    # We don't need gradients on to do reporting
    model.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(validation_loader):
        vinputs, vlabels = vdata
        voutputs = model(vinputs)
        accuracy = torch.sum(1 * (voutputs.argmax(dim=1) == vlabels)) /(voutputs.shape[1])
        print(accuracy)
        vloss = loss_fn(voutputs, vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))


    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, (3, 3),padding="same")
        self.conv2 = nn.Conv2d(32, 64, (3,3),padding='same')
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d((2,2), stride=(2,2))
        self.fc = nn.Linear(16384, 21)
        self.dropout = nn.Dropout(p=0.5)
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.flatten(x) # flatten all dimensions except batch
        x = self.fc(x)
        x = self.dropout(x)
        x = self.softmax(x)
        
        return x
    

In [ ]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)

output = loss(input, target)
output.backward()
